In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch import nn
from tqdm import tqdm
from datetime import date, timedelta

import warnings
warnings.filterwarnings('ignore')


In [2]:
data = pd.DataFrame({'X1':[-i for i in range(500)],
                  'X2':np.random.randint(low=10,high=20,size=500),
                  'X3':np.random.normal(size=500),
                  'Y':np.sin(np.linspace(0, 2*np.pi, 500))})
data.describe()


,X1,X2,X3,Y
count,500.000000,500.000000,500.000000,5.000000e+02
mean,-249.500000,14.502000,0.040361,5.684342e-17
std,144.481833,2.819556,0.951013,7.071068e-01
min,-499.000000,10.000000,-3.103284,-9.999950e-01
25%,-374.250000,12.000000,-0.565710,-7.048748e-01
50%,-249.500000,14.000000,0.004059,-1.224647e-16
75%,-124.750000,17.000000,0.648191,7.048748e-01
max,0.000000,19.000000,2.892125,9.999950e-01


In [4]:
data


,X1,X2,X3,Y
0,0,13,1.318223,0.000000e+00
1,-1,13,0.123312,1.259122e-02
2,-2,18,0.044109,2.518045e-02
3,-3,10,1.638628,3.776568e-02
4,-4,18,-0.405989,5.034492e-02
...,...,...,...,...
495,-495,13,0.266171,-5.034492e-02
496,-496,12,-0.211158,-3.776568e-02
497,-497,12,0.911154,-2.518045e-02
498,-498,14,-0.815399,-1.259122e-02


In [11]:
# 创建时间步为25的滑动窗口
window_size = 25
num_samples = len(data) - window_size + 1

X_samples = []
Y_samples = []

for i in range(num_samples):
    X_sample = data.iloc[i:i + window_size][['X1', 'X2', 'X3']].values
    Y_sample = data.iloc[i + window_size - 1]['Y']
    X_samples.append(X_sample)
    Y_samples.append(Y_sample)

X_samples = np.array(X_samples)
Y_samples = np.array(Y_samples)



# 现在X_samples和Y_samples包含了时间步为25的样本数据
# X_samples的形状为(num_samples, 25, num_features)，Y_samples的形状为(num_samples,)

# 4. 划分训练集和测试集（可选）
# 如果您需要划分训练集和测试集，请使用train_test_split函数



In [17]:
print('X_samples.shape',X_samples.shape)
print('Y_samples.shape',Y_samples.shape)


X_samples.shape (476, 25, 3)
Y_samples.shape (476,)


In [15]:
X_train,X_val,y_train,y_val = train_test_split(X_samples,
                                               Y_samples,test_size=0.2,
                                               shuffle=False)
X_train.shape


(380, 25, 3)

In [ ]:
import torch
import torch.nn as nn

# 定义循环神经网络模型
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # 初始化隐藏状态
        h0 = torch.zeros(1, x.size(0), self.hidden_size)
        # 前向传播 RNN
        out, _ = self.rnn(x, h0)
        # 取最后一个时间步
        out = out[:, -1, :]
        # 通过全连接层
        out = self.fc(out)
        return out

# 实例化模型
input_size = 3  # 特征 X1, X2, X3
hidden_size = 50
output_size = 1  # 预测 Y
model = RNN(input_size, hidden_size, output_size)
